<a href="https://colab.research.google.com/github/jimmyshah83/deep_learning/blob/master/GAN_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

GAN Using Keras

In [0]:
#from google.colab import drive
#drive.mount('/content/drive')

In [0]:
#import zipfile
#import io
#from zipfile import ZipFile
#with ZipFile('drive/My Drive/cropped.zip', 'r') as zf:
#   zf.extractall('drive/My Drive/images/')

In [0]:
import tensorflow.contrib.eager as tfe

In [0]:
# Hyperparameters
IMAGE_SIZE = 64
NOISE_SIZE = 100
LR_D = 0.00004
LR_G = 0.0004
BATCH_SIZE = 32
EPOCHS = 1
BETA1 = 0.5
WEIGHT_INIT_STDDEV = 0.02
EPSILON = 0.00005
SAMPLES_TO_SHOW = 5
BUFFER_SIZE = 9877

In [0]:
import os
import time
import numpy as np
from glob import glob
import datetime
import random
from PIL import Image
import matplotlib.pyplot as plt
from __future__ import absolute_import, division, print_function

import tensorflow as tf
from keras.models import Model, Sequential
from keras.layers import *
from keras.optimizers import Adam
from keras.initializers import TruncatedNormal

%matplotlib inline

In [0]:
def get_batches(data):
    batches = []
    for i in range(int(data.shape[0]//BATCH_SIZE)):
        batch = data[i * BATCH_SIZE:(i + 1) * BATCH_SIZE]
        augmented_images = []
        for img in batch:
            image = Image.fromarray(img)
            if random.choice([True, False]):
                image = image.transpose(Image.FLIP_LEFT_RIGHT)
            augmented_images.append(np.asarray(image))
        batch = np.asarray(augmented_images)
        normalized_batch = (batch / 127.5) - 1.0
        batches.append(normalized_batch)
    return batches

In [428]:
train_images = np.asarray([np.asarray(Image.open(file).resize((IMAGE_SIZE, IMAGE_SIZE))) 
                           for file in glob('drive/My Drive/images/' + '10*.png')])

print ("Input: " + str(train_images.shape))

Input: (111, 64, 64, 3)


In [0]:
#train_images = (train_images - 127.5) / 127.5 # Normalize the images to [-1, 1]
# Batch and shuffle the data
#train_dataset = tf.data.Dataset.from_tensor_slices(train_images)
#train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
train_dataset = get_batches(train_images)

In [0]:
def make_generator_model():
    model = Sequential()
    
    model.add(Dense(8*8*1024, input_shape=(100,)))
    model.add(Reshape((8, 8, 1024)))
    model.add(LeakyReLU())
    
    model.add(Conv2DTranspose(512, (5, 5), padding='same', strides=2, kernel_initializer=TruncatedNormal(stddev=WEIGHT_INIT_STDDEV)))
    model.add(BatchNormalization(epsilon=EPSILON))
    model.add(LeakyReLU())
    
    
    model.add(Conv2DTranspose(256, (5, 5), padding='same', strides=2, kernel_initializer=TruncatedNormal(stddev=WEIGHT_INIT_STDDEV)))
    model.add(BatchNormalization(epsilon=EPSILON))
    model.add(LeakyReLU())
    
    model.add(Conv2DTranspose(128, (5, 5), padding='same', strides=2, kernel_initializer=TruncatedNormal(stddev=WEIGHT_INIT_STDDEV)))
    model.add(BatchNormalization(epsilon=EPSILON))
    model.add(LeakyReLU())
    
    model.add(Conv2DTranspose(64, (5, 5), padding='same', strides=2, kernel_initializer=TruncatedNormal(stddev=WEIGHT_INIT_STDDEV)))
    model.add(BatchNormalization(epsilon=EPSILON))
    model.add(LeakyReLU())
    
    model.add(Conv2DTranspose(3, (5, 5), padding='same', strides=2, kernel_initializer=TruncatedNormal(stddev=WEIGHT_INIT_STDDEV)))
    
    return model

In [0]:
generator = make_generator_model()

noise = tf.random.normal([1, 100])
generated_image = generator(noise)

In [0]:
def make_discriminator_model():
  model = Sequential()
  model.add(Conv2D(64, (5, 5), padding='same', strides=2, kernel_initializer=TruncatedNormal(stddev=WEIGHT_INIT_STDDEV)))
  model.add(BatchNormalization(epsilon=EPSILON))
  model.add(LeakyReLU())
    
  model.add(Conv2D(128, (5, 5), padding='same', strides=2, kernel_initializer=TruncatedNormal(stddev=WEIGHT_INIT_STDDEV)))
  model.add(BatchNormalization(epsilon=EPSILON))
  model.add(LeakyReLU())
    
  model.add(Conv2D(256, (5, 5), padding='same', strides=2, kernel_initializer=TruncatedNormal(stddev=WEIGHT_INIT_STDDEV)))
  model.add(BatchNormalization(epsilon=EPSILON))
  model.add(LeakyReLU())
              
  model.add(Conv2D(512, (5, 5), padding='same', strides=2, kernel_initializer=TruncatedNormal(stddev=WEIGHT_INIT_STDDEV)))
  model.add(BatchNormalization(epsilon=EPSILON))
  model.add(LeakyReLU())
              
  model.add(Conv2D(1024, (5, 5), padding='same', strides=2, kernel_initializer=TruncatedNormal(stddev=WEIGHT_INIT_STDDEV)))
  model.add(BatchNormalization(epsilon=EPSILON))
  model.add(LeakyReLU())
              
  model.add(Flatten())
  model.add(Dense(1, activation='sigmoid'))
              
  return model

In [433]:
discriminator = make_discriminator_model()
decision = discriminator(generated_image)
print (decision)

Tensor("sequential_47/dense_47_1/Sigmoid:0", shape=(1, 1), dtype=float32)


In [0]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [0]:
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

In [0]:
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

In [0]:
discriminator_optimizer = Adam(lr=LR_D, beta_1=BETA1)
generator_optimizer = Adam(lr=LR_G, beta_1=BETA1)

In [0]:
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, NOISE_SIZE])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
      generated_images = generator(noise)

      real_output = discriminator(images)
      fake_output = discriminator(generated_images)

      gen_loss = generator_loss(fake_output)
      disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

In [0]:
def train(dataset):  
  for epoch in range(EPOCHS):
    start = time.time()
    
    for image_batch in dataset:
      train_step(image_batch)
    
    print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

In [440]:
train(train_dataset)

ValueError: ignored